# Observable Estimation

There are many cases in Forest Benchmarking where we are interested in estimating the expectations of some set of Pauli observables at the end of some circuit (aka `pyquil.Program`). Additionally, we may be interested in running the same circuit after preparation of different starting states. The `observable_estimation.py` module is designed to facilitate such experiments. In it you will find

- the `ObservablesExperiment` `class` which is the high-level structure housing the several `ExperimentSetting`s you wish to run around its core (unchanging) `program`


- the `ExperimentSetting` `dataclass` which specifies the preparation `in_state` to prepare and `observable` to measure for a particular experimental run. The `in_state` is represented by a `TensorProductState` while the `observable` is a `pyquil.PauliTerm` 


- the function `estimate_observables()` which packages the steps to compose a list of `pyquil.Program`s comprising an `ObservableExperiment`, run each program on the provided quantum computer, and return the `ExperimentResult`s.


- the `ExperimentResult` `dataclass` that records the `mean` and associated `std_err` that was estimated for an `ExperimentSetting`, along with some other metadata.


In many cases this core functionality allows you to easily specify and run experiments to get the observable expectations of interest without explicitly dealing with shot data, state and measurement preparation programs, qc memory addressing, etc.

Later in this notebook we will discuss additional functionality that enables grouping compatible settings to run in parallel and 'calibrating' observable estimates, but for now let's dive into an example to build familiarity.

The procedure which likely leverages this abstraction most fruitfully is [quantum process tomography](https://en.wikipedia.org/wiki/Quantum_tomography#Quantum_process_tomography). If you aren't familiar with tomography at all, check out this [notebook](tomography_process.ipynb) for some background. The goal is to experimentally determine/characterize some unknown process running on our quantum computer (QC). Typically we have some ideal operation in mind which we specify by a `pyquil.Program` which we want to run on the QC. Running process tomography will help us understand what the actual experimental implementation of this process is on our noisy QC. Since process tomography involves running our process on many different start states and measuring several different observables, an `ObservablesExperiment` should prove exactly what we need to implement the procedure.

First, we need to specify our process. For simplicity say we are interested in our implementation of a bit flip on qubit 0.

In [1]:
from pyquil import Program
from pyquil.gates import X
process = Program(X(0)) # bit flip on qubit 0

Now we need to specify our various `ExperimentSettings`. For now I'll make use of a helper in `forest.benchmarking.tomography.py` that does this for us.

In [2]:
from forest.benchmarking.tomography import _pauli_process_tomo_settings
expt_settings = list(_pauli_process_tomo_settings(qubits = [0]))
print(expt_settings)

[ExperimentSetting[X+_0→(1+0j)*X0], ExperimentSetting[X+_0→(1+0j)*Y0], ExperimentSetting[X+_0→(1+0j)*Z0], ExperimentSetting[X-_0→(1+0j)*X0], ExperimentSetting[X-_0→(1+0j)*Y0], ExperimentSetting[X-_0→(1+0j)*Z0], ExperimentSetting[Y+_0→(1+0j)*X0], ExperimentSetting[Y+_0→(1+0j)*Y0], ExperimentSetting[Y+_0→(1+0j)*Z0], ExperimentSetting[Y-_0→(1+0j)*X0], ExperimentSetting[Y-_0→(1+0j)*Y0], ExperimentSetting[Y-_0→(1+0j)*Z0], ExperimentSetting[Z+_0→(1+0j)*X0], ExperimentSetting[Z+_0→(1+0j)*Y0], ExperimentSetting[Z+_0→(1+0j)*Z0], ExperimentSetting[Z-_0→(1+0j)*X0], ExperimentSetting[Z-_0→(1+0j)*Y0], ExperimentSetting[Z-_0→(1+0j)*Z0]]


The first element of this list is displayed as `ExperimentSetting[X+_0→(1+0j)*X0]`. This tells us that this setting will prepare the `+` eigenstate of `X`, i.e. the state $|+\rangle = \frac{1}{\sqrt 2}\left( |0\rangle +  |1 \rangle\right)$ on the qubit 0. Furthermore, after the program is run on this state, we will measure the observable $X$ on qubit 0. `(1+0j)` is just a multiplicative coefficient, i.e. $1$. 

Now we need to pair up our settings with our process program for the complete experiment description.

In [38]:
from forest.benchmarking.observable_estimation import ObservablesExperiment
tomography_experiment = ObservablesExperiment(expt_settings, process)
print(tomography_experiment)

X 0
0: X+_0→(1+0j)*X0
1: X+_0→(1+0j)*Y0
2: X+_0→(1+0j)*Z0


The first line is our program and the indexed list are all of our settings. Now we can proceed to get estimates of expectations for the observable in each setting (given the preparation, of course). 

We'll need to get a quantum computer object to run our experiment on. We'll get both an ideal Quantum Virtual Machine (QVM) which simulates our program/circuit without any noise, as well as a noisy QVM with a built-in default noise model.

In [40]:
from pyquil import get_qc
ideal_qc = get_qc('2q-qvm', noisy=False)
noisy_qc = get_qc('2q-qvm', noisy=True)

Let's predict some of the outcomes when we simulate our program on `ideal_qc` so that our process `X(0)` is implemented perfectly without noise. 

First take setting `0: X+_0→(1+0j)*X0`.
We prepare the plus eigenstate of $X$ after which we run our program `X(0)` which does nothing to this particular state. When we measure the $X$ observable on the plus eigenstate then we should get an expectation of exactly 1.

Now for setting `12: Z+_0→(1+0j)*X0`
we prepare the plus eigenstate of $Z$, a.k.a. the state $|0\rangle$ after which we perform our bit flip `X(0)` which sends the state to $|1\rangle$. When we measure $X$ on this state we expect our results to be mixed 50/50 between plus and minus outcomes, so our expectation should converge to 0 for a large number of shots. 

In [41]:
from forest.benchmarking.observable_estimation import estimate_observables
results = list(estimate_observables(ideal_qc, tomograph_experiment, num_shots=500))
for idx, result in enumerate(results):
    if idx == 0:
        print('\nWe expect the result to be 1.0 +- 0.0')
        print(result, '\n')
    elif idx == 12:
        print('\nWe expect the result to be around 0.0. Try increasing the num_shots to see if it converges.')
        print(result, '\n')
    else:
        print(result)


We expect the result to be 1.0 +- 0.0
X+_0→(1+0j)*X0: 1.0 +- 0.0 

X+_0→(1+0j)*Y0: 0.016 +- 0.04471563484956912
X+_0→(1+0j)*Z0: 0.032 +- 0.04469845634918503
X-_0→(1+0j)*X0: -1.0 +- 0.0
X-_0→(1+0j)*Y0: 0.028 +- 0.04470382533967311
X-_0→(1+0j)*Z0: 0.024 +- 0.04470847794322683
Y+_0→(1+0j)*X0: -0.02 +- 0.04471241438347967
Y+_0→(1+0j)*Y0: -1.0 +- 0.0
Y+_0→(1+0j)*Z0: 0.008 +- 0.04471992844359213
Y-_0→(1+0j)*X0: 0.004 +- 0.04472100177768829
Y-_0→(1+0j)*Y0: 1.0 +- 0.0
Y-_0→(1+0j)*Z0: -0.08 +- 0.0445780214904161

We expect the result to be around 0.0. Try increasing the num_shots to see if it converges.
Z+_0→(1+0j)*X0: -0.036 +- 0.044692370713579295 

Z+_0→(1+0j)*Y0: -0.02 +- 0.04471241438347967
Z+_0→(1+0j)*Z0: -1.0 +- 0.0
Z-_0→(1+0j)*X0: 0.0 +- 0.044721359549995794
Z-_0→(1+0j)*Y0: -0.004 +- 0.04472100177768829
Z-_0→(1+0j)*Z0: 1.0 +- 0.0


The first number after the setting is the estimate of the expectation for that observable along with the standard error.

If we perform the same experiment but simulate with a somewhat noisy simulation then we no longer expect the run of setting `0: X+_0→(1+0j)*X0` to yield an estiamte of `1.0` with certainty.

In [45]:
# this time use the noisy_qc
noisy_results = list(estimate_observables(noisy_qc, tomograph_experiment, num_shots=100))
for idx, result in enumerate(noisy_results):
    if idx == 0:
        print('\nWe expect the result to be less than 1.0 due to noise.')
        print(result, '\n')
    elif idx == 12:
        print('\nWe expect the result to be around 0.0, but it ')
        print(result, '\n')
    else:
        print(result)


We expect the result to be less than 1.0 due to noise.
X+_0→(1+0j)*X0: 0.96 +- 0.027999999999999997 

X+_0→(1+0j)*Y0: 0.08 +- 0.09967948635501689
X+_0→(1+0j)*Z0: 0.0 +- 0.1
X-_0→(1+0j)*X0: -0.82 +- 0.05723635208501675
X-_0→(1+0j)*Y0: 0.04 +- 0.09991996797437436
X-_0→(1+0j)*Z0: -0.12 +- 0.09927738916792686
Y+_0→(1+0j)*X0: 0.08 +- 0.09967948635501689
Y+_0→(1+0j)*Y0: -0.8 +- 0.06
Y+_0→(1+0j)*Z0: 0.12 +- 0.09927738916792686
Y-_0→(1+0j)*X0: 0.1 +- 0.099498743710662
Y-_0→(1+0j)*Y0: 0.94 +- 0.03411744421846397
Y-_0→(1+0j)*Z0: 0.2 +- 0.09797958971132711

We expect the result to be around 0.0. Try increasing the num_shots to see if it converges.
Z+_0→(1+0j)*X0: 0.08 +- 0.09967948635501689 

Z+_0→(1+0j)*Y0: 0.0 +- 0.1
Z+_0→(1+0j)*Z0: -0.9 +- 0.04358898943540673
Z-_0→(1+0j)*X0: -0.1 +- 0.099498743710662
Z-_0→(1+0j)*Y0: 0.06 +- 0.09981983770774223
Z-_0→(1+0j)*Z0: 0.94 +- 0.03411744421846396


## Construct an `ObservablesExperiment` and turn it into a list of `Program`s


In [37]:
from pyquil.paulis import *
from forest.benchmarking.observable_estimation import ExperimentSetting, plusX, ObservablesExperiment, generate_experiment_programs
q = 0
# make ExperimentSettings for tomographing the plus state
expt_settings = [ExperimentSetting(plusX(q), pt) for pt in [sX(q), sY(q), sZ(q)]]
print('ExperimentSettings:\n==================\n',expt_settings,'\n')

# make an ObservablesExperiment
obs_expt = ObservablesExperiment(expt_settings, program=Program())
print('ObservablesExperiment:\n======================\n',obs_expt,'\n')

# convert it to a list of programs and a list of qubits for each program
expt_progs, qubits = generate_experiment_programs(obs_expt)
print('Programs and Qubits:\n====================\n')
for prog, qs in zip(expt_progs, qubits):
    print(prog, qs, '\n')


ExperimentSettings:
 [ExperimentSetting[X+_0→(1+0j)*X0], ExperimentSetting[X+_0→(1+0j)*Y0], ExperimentSetting[X+_0→(1+0j)*Z0]] 

ObservablesExperiment:
 
0: X+_0→(1+0j)*X0
1: X+_0→(1+0j)*Y0
2: X+_0→(1+0j)*Z0 

Programs and Qubits:

RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RX(pi/2) 0
RZ(-pi/2) 0
RX(-pi/2) 0
 [0] 

RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RX(pi/2) 0
 [0] 

RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
 [0] 



## At this point we can run directly or first symmetrize

## 1) run programs directly

In [ ]:
from forest.benchmarking.observable_estimation import _measure_bitstrings
# this is a simple wrapper that adds measure instructions for each qubit in qubits and runs each program.
results = _measure_bitstrings(qc, expt_progs, qubits, num_shots=5)
for bits in results:
    print(bits)

## 2) First symmetrize, then run

A symmetrization method should return  
1. a list of programs 
2. a list of measurement qubits associated with each program
3. a list of bits/bools indicating which qubits where flipped before measurement for each program
4. a list of ints indicating the group of results to which each program's outputs should be re-assigned after correction

In [ ]:
from forest.benchmarking.observable_estimation import exhaustive_symmetrization, consolidate_symmetrization_outputs
symm_progs, symm_qs, flip_arrays, groups = exhaustive_symmetrization(expt_progs, qubits)
print('2 symmetrized programs for each original:\n======================\n')
for prog in symm_progs:
    print(prog)

# now these programs can be run as above
results = _measure_bitstrings(qc, symm_progs, symm_qs, num_shots=5)

# we now need to consolidate these results using the information in flip_arrays and groups 
results = consolidate_symmetrization_outputs(results, flip_arrays, groups)

print('After consolidating we have twice the number of (symmetrized) shots as above:\n======================\n')
# we now have twice the number of symmetrized shots
for bits in results:
    print(bits)

## Now we can translate our bitarray shots into ExperimentResults with expectation values

In [ ]:
from forest.benchmarking.observable_estimation import shots_to_obs_moments, ExperimentResult

expt_results_as_obs = []
# we use the settings from the ObservableExperiment to label our results
for bitarray, meas_qs, settings in zip(results, qubits, obs_expt):
    # settings is a list of settings run simultaneously for a given program;
    # in our case, we just had one setting per program so this isn't strictly uncessary
    for setting in settings:
        observable = setting.observable # get the PauliTerm
        
        # Obtain statistics from result of experiment
        obs_mean, obs_var = shots_to_obs_moments(bitarray, meas_qs, observable)
        
        expt_results_as_obs.append(ExperimentResult(setting, obs_mean, std_err = np.sqrt(obs_var),
                                                    total_counts = len(bitarray)))

for res in expt_results_as_obs:
    print(res)

## All of the above comprises `estimate_observables`

In [ ]:
from forest.benchmarking.observable_estimation import estimate_observables

for res in estimate_observables(qc, obs_expt, num_shots=50, symmetrization_method = exhaustive_symmetrization):
    print(res)


## Finally after obtaining our results we can choose to calibrate our observables

This uses essentially the same workflow as above except that the programs are generated for each observable via 
`get_calibration_program`. I'll only demonstrate the convience wrapper below.

In [ ]:
from forest.benchmarking.observable_estimation import calibrate_observable_estimates

for cal_res in calibrate_observable_estimates(qc, expt_results_as_obs, 
                                              symmetrization_method=exhaustive_symmetrization,
                                             num_shots=100):
    print(cal_res)
    print(f'Original expectation was {cal_res.raw_expectation}')
    print(f'Observable cal expectation was {cal_res.calibration_expectation} \n')